In [1]:
from cil.io import NEXUSDataReader, NEXUSDataWriter
from cil.optimisation.algorithms import PDHG
from cil.processors import Slicer
from cil.plugins.astra.operators import ProjectionOperator
from cil.optimisation.functions import IndicatorBox, L2NormSquared, MixedL21Norm, BlockFunction
from cil.optimisation.operators import GradientOperator, BlockOperator
from cil.optimisation.operators.GradientOperator import Gradient_C
from cil.framework import BlockDataContainer, AcquisitionGeometry
import matplotlib.pyplot as plt
import pickle

In [2]:
# Load data after the RingRemover processor
name = "data_after_ring_remover_318_398.nxs"
read_data = NEXUSDataReader(file_name = "HyperspectralData/" + name)
data = read_data.load_data()

# Select only 5 energy channels. To reproduce the results in the paper, select 318-398 energy interval
data = Slicer(roi={'channel': (37,42),'vertical': (17,22)})(data)
print(data.shape)

(5, 5, 120, 400)


In [3]:
# Extract geometry and angle information from data
ag = data.geometry
ig = ag.get_ImageGeometry()
angles = ag.angles

# For every geometry in list_geometries, define operators A_i
Aop = ProjectionOperator(ig, ag, 'gpu')
Grad = GradientOperator(ig, correlation="SpaceChannels")
K = BlockOperator(Aop, Grad)

# Regularisation parameter
alpha = 0.002

# List of BlockFunctions
f = BlockFunction(0.5*L2NormSquared(b = data), alpha * MixedL21Norm())

# Positivity constraint of G
g = IndicatorBox(lower=0)

# Run SPDHG algorithm
normK = K.norm()
sigma = 1./normK
tau = 1./normK

Initialised GradientOperator with C backend running with  16  threads


In [4]:
def callback_save(iteration, objective_value, solution):
    """Callback function to save images"""
    
    list_iter = [z for z in range(0,51,5)]+[1000]
    
    if iteration in list_iter:
        name = "pdhg_spatiospectral_tv_reconstruction_iter_{}.nxs".format(iteration)
        writer = NEXUSDataWriter(file_name = "HyperspectralData/" + name,
                                 data = solution)     
        writer.write() 

In [5]:
pdhg = PDHG(f = f, g = g, operator = K, 
              max_iteration = 1001,
            sigma=sigma, tau=tau)
pdhg.run(verbose=0, callback = callback_save) 

pdhg_info = {}
pdhg_info['timing'] = pdhg.timing
with open('HyperspectralData/objectives_pdhg_spatiospectral_tv.pkl','wb') as f:
    pickle.dump(pdhg_info, f)

PDHG setting up
PDHG configured


/home/vaggelis/miniconda3/envs/cil21_tigre_astra/lib/python3.8/site-packages/cil/framework/framework.py:2185: RuntimeWarning: invalid value encountered in true_divide
  pwop(self.as_array(), x2.as_array(), *args, **kwargs )
